# PLC figures.
- Generate data,
- and prev plots for 'optimal PLC' figures.

In [7]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import Range1d
from bokeh.io import export_png
output_notebook()

import numpy as np

from voltagebudget import util
from voltagebudget import neurons
from voltagebudget import plc
from voltagebudget.util import locate_firsts
from voltagebudget.util import locate_peaks
from voltagebudget.util import filter_voltages

# -
def create_traces(ts_x, changes, fn):
    d_variances = []
    errors = []
    for c in changes:
        initial, target, obs, ts_opt = fn(ts_x, c)

#         d_v = initial - obs 
        d_v = obs 
        err = util.mae(ts_x, ts_opt)

        d_variances.append(d_v)
        errors.append(err)

    d_variances = np.asarray(d_variances)
    errors = np.asarray(errors)

    return d_variances, errors


Loading BokehJS ...

In [8]:
changes = np.linspace(0.0, 0.99, 50)

t=0.5
stim_number=40
stim_onset=0.2
stim_offset=0.250
stim_rate=8
seed_stim=7525
time_step=1e-5

ns_x, ts_x = util.poisson_impulse(
        t,
        stim_onset,
        stim_offset - stim_onset,
        stim_rate,
        dt=time_step,
        n=stim_number,
        seed=seed_stim)

In [9]:
p = figure(plot_width=400, plot_height=200)
p.circle(ts_x, ns_x, color="grey", alpha=0.3)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# Max deviants

In [25]:
percent_change = 0.05

initial, target, obs, ts_opt = plc.max_deviant(ts_x, percent_change)

p = figure(plot_width=400, plot_height=200)
p.circle(ts_opt, ns_x, color="black")
p.circle(ts_x, ns_x, color="grey", alpha=0.3)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0.2, 0.25)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

print(initial, target, obs)

(0.012855946336618894, 0.012213149019787949, 0.012211501892174523)


In [26]:
# Save for fig
X = np.vstack([ns_x, ts_x, ts_opt])
header = "n,ts,ts_opt"
np.savetxt("../analysis/plc_spikes.csv", X.T, delimiter=",", header=header, comments="")

In [12]:
# -
d_variances_ma, errors_ma = create_traces(ts_x, changes, plc.max_deviant)

p = figure(plot_width=300, plot_height=200)
p.line(x=d_variances_ma, y=errors_ma, color="black")
p.xaxis.axis_label = 'Variance (ms)'
p.yaxis.axis_label = 'Error (ms)'
# p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [13]:
# Save for fig
X = np.vstack([d_variances_ma, errors_ma, changes])
header = "variances,errors,percent_changes"
np.savetxt("../analysis/plc_traces.csv", X.T, delimiter=",", header=header, comments="")